In [8]:
import mediapipe as mp
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from mpl_toolkits.mplot3d import Axes3D

In [9]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

# mediapipe_part

미디어 파이프를 활용해 점을 찍고 그 점들을 활용한 데이터들을 반환함.


In [3]:
mediapipe_landmark = [
    'nose',
    'left_eye_inner',
    'left_eye',
    'left_eye_outer',
    'right_eye_inner',
    'right_eye',
    'right_eye_outer',
    'left_ear',
    'right_ear',
    'mouth_left',
    'mouth_right',
    'left_shoulder',
    'right_shoulder',
    'left_elbow',
    'right_elbow',
    'left_wrist', 'right_wrist',
    'left_pinky_1', 'right_pinky_1',
    'left_index_1', 'right_index_1',
    'left_thumb_2', 'right_thumb_2',
    'left_hip', 'right_hip',
    'left_knee', 'right_knee',
    'left_ankle', 'right_ankle',
    'left_heel', 'right_heel',
    'left_foot_index', 'right_foot_index',
]

landmark_forDot = []

for i in mediapipe_landmark:
    for j in ["_x", "_y", "_z"]:
        landmark_forDot.append(i + j)

In [107]:
def getMediapipeData(video, Z_ignore = True, size_estimation=True):
    cap = cv2.VideoCapture(video)
    width, height = cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT
    baseData = coording_points(cap)
    if Z_ignore is True:
        drop_Z_coords(baseData)
        
    interData = pd.DataFrame(index=range(len(baseData.index)))
    
    if size_estimation is True:
        push_realized_sizes(baseData, interData, width, height)
    
    push_hip_angle(baseData, interData)
    push_knee_angle(baseData, interData)
    push_ankle_angle(baseData, interData)
    
    return interData

# 기본 landmark 좌표 추출

In [80]:
def coording_points(video):
    results_points = []
    cap = video
    with mp_pose.Pose(
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                # print("카메라를 찾을 수 없습니다.")
                break

            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = pose.process(image)
            points = []
            for i in range(len(mediapipe_landmark)):
                points.append(results.pose_world_landmarks.landmark[i].x)
                points.append(results.pose_world_landmarks.landmark[i].y)
                points.append(results.pose_world_landmarks.landmark[i].z)
            results_points.append(points)

            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            mp_drawing.draw_landmarks(
                image,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
            cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
            if cv2.waitKey(5) & 0xFF == 27:
                break
    cv2.destroyAllWindows
    cap.release()
    return pd.DataFrame(columns = landmark_forDot, data = results_points)
    
    

In [19]:
def drop_Z_coords(data):
    for landmark in mediapipe_landmark:
        data.drop(landmark+"_z", inplace = True, axis=1)

# 2차원 좌표, 벡터, 각도 측정

z좌표는 신뢰도가 떨어진다는 피드백이 있었음.
z 좌표의 신뢰도를 측정할 수 있었다면 어떨까? 싶지만
일단 완전한 옆모습만을 찍는다는 것을 근거로 z좌표를 제외한 값들을 사용하기로 함.

In [38]:
#2차원 좌표 얻음
#pandas framedata, 랜드마크 이름, 몇번째 원소인가

def get_2D_Coord(Data, landName, row):
    coord = np.array([
        Data.loc[row, landName+"_x"],
        Data.loc[row, landName+"_y"]
    ])
    
    return coord

In [39]:
def get_2D_Vector(data, land1, land2, row):
    coord1 = get_2D_Coord(data, land1, row)
    coord2 = get_2D_Coord(data, land2, row)
    
    vector = np.array(coord1 - coord2)
    return vector

In [104]:
def push_realized_sizes(data, interData, width, height):
    w = width
    h = height
    data_length = len(data)
    
    Lthigh_sizes = []
    Lshin_sizes = []
    Lfoot_sizes = []
    Rthigh_sizes = []
    Rshin_sizes = []
    Rfoot_sizes = []
    
    for i in range(data_length):
        left_thigh = get_2D_Vector(data, "left_hip", "left_knee", i)
        left_thigh[0] *= width
        left_thigh[1] *= height
        left_thigh_size = np.linalg.norm(left_thigh)
        Lthigh_sizes.append(left_thigh_size)
        
        left_shin = get_2D_Vector(data, "left_knee", "left_ankle", i)
        left_shin[0] *= width
        left_shin[1] *= height
        left_shin_size = np.linalg.norm(left_shin)
        Lshin_sizes.append(left_shin_size)
        
        left_foot = get_2D_Vector(data, "left_ankle", "left_foot_index",i)
        left_foot[0] *= width
        left_foot[1] *= height
        left_foot_size = np.linalg.norm(left_foot)
        Lfoot_sizes.append(left_foot_size)
        
        right_thigh = get_2D_Vector(data, "right_hip", "right_knee", i)
        right_thigh[0] *= width
        right_thigh[1] *= height
        right_thigh_size = np.linalg.norm(right_thigh)
        Rthigh_sizes.append(right_thigh_size)
        
        right_shin = get_2D_Vector(data, "right_knee", "right_ankle", i)
        right_shin[0] *= width
        right_shin[1] *= height
        right_shin_size = np.linalg.norm(right_shin)
        Rshin_sizes.append(right_shin_size)
        
        right_foot = get_2D_Vector(data, "right_ankle", "right_foot_index",i)
        right_foot[0] *= width
        right_foot[1] *= height
        right_foot_size = np.linalg.norm(right_foot)
        Rfoot_sizes.append(right_foot_size)
        
    interData["left_thigh_size"] = Lthigh_sizes
    interData["left_shin_size"] = Lshin_sizes
    interData["left_foot_size"] = Lfoot_sizes
    interData["right_thigh_size"] = Rthigh_sizes
    interData["right_shin_size"] = Rshin_sizes
    interData["right_foot_size"] = Rfoot_sizes
            
        
        

In [71]:
def push_hip_angle(data, interData):
    data_length = len(data)
    Rhip_angles = []
    Lhip_angles = []
    for i in range(data_length):
        left_shoulder = get_2D_Vector(data, "left_shoulder", "left_hip", i)
        left_knee = get_2D_Vector(data, "left_knee", "left_hip", i)
        
        ldot = np.dot(left_shoulder, left_knee)
        lcos = ldot / (np.linalg.norm(left_shoulder) * np.linalg.norm(left_knee))
        left_hip_angle = np.arccos(lcos)
        
        right_shoulder = get_2D_Vector(data, "right_shoulder", "right_hip", i)
        right_knee = get_2D_Vector(data, "right_knee", "right_hip", i)
        
        rdot = np.dot(right_shoulder, right_knee)
        rcos = rdot / (np.linalg.norm(right_shoulder) * np.linalg.norm(right_knee))
        right_hip_angle = np.arccos(rcos)
        
        Rhip_angles.append(right_hip_angle)
        Lhip_angles.append(left_hip_angle)
        
    interData["right_hip_angle"] = Rhip_angles
    interData["left_hip_angle"] = Lhip_angles
        

In [75]:
def push_knee_angle(data, interData):
    data_length = len(data)
    Rknee_angles = []
    Lknee_angles = []
    for i in range(data_length):
        left_hip = get_2D_Vector(data, "left_hip", "left_knee", i)
        left_ankle = get_2D_Vector(data, "left_ankle", "left_knee", i)
        
        ldot = np.dot(left_hip, left_ankle)
        lcos = ldot / (np.linalg.norm(left_hip) * np.linalg.norm(left_ankle))
        left_knee_angle = np.arccos(lcos)
        
        right_hip = get_2D_Vector(data, "right_hip", "right_knee", i)
        right_ankle = get_2D_Vector(data, "right_ankle", "right_knee", i)
        
        rdot = np.dot(right_hip, right_ankle)
        rcos = rdot / (np.linalg.norm(right_hip) * np.linalg.norm(right_ankle))
        right_knee_angle = np.arccos(rcos)
        
        Rknee_angles.append(right_knee_angle)
        Lknee_angles.append(left_knee_angle)
    
    interData["right_knee_angle"] = Rknee_angles
    interData["left_knee_angle"] = Lknee_angles
        

In [89]:
def push_ankle_angle(data, interData):
    data_length = len(data)
    Rankle_angles = []
    Lankle_angles = []
    
    for i in range(data_length):
        left_knee = get_2D_Vector(data, "left_knee", "left_ankle", i)
        left_heel = get_2D_Vector(data, "left_heel", "left_foot_index", i)

        ldot = np.dot(left_knee, left_heel)
        lcos = ldot / (np.linalg.norm(left_knee) * np.linalg.norm(left_heel))
        left_ankle_angle = np.arccos(lcos)

        right_knee = get_2D_Vector(data, "right_knee", "right_ankle", i)
        right_heel = get_2D_Vector(data, "right_heel", "right_foot_index", i)

        rdot = np.dot(right_knee, right_heel)
        rcos = rdot / (np.linalg.norm(right_knee) * np.linalg.norm(right_heel))
        right_ankle_angle = np.arccos(rcos)
    
        Rankle_angles.append(right_ankle_angle)
        Lankle_angles.append(left_ankle_angle)
        
    interData["right_ankle_angle"] = Rankle_angles
    interData["left_ankle_angle"] = Lankle_angles

# ForTest

In [108]:
data = getMediapipeData("walk_side.mp4")

In [109]:
data

,left_thigh_size,left_shin_size,left_foot_size,right_thigh_size,right_shin_size,right_foot_size,right_hip_angle,left_hip_angle,right_knee_angle,left_knee_angle,right_ankle_angle,left_ankle_angle
0,1.587078,1.450979,0.499202,1.551623,1.489441,0.458296,3.129419,3.024758,2.935776,3.109309,1.480348,1.465436
1,1.580843,1.427614,0.478772,1.556061,1.481528,0.428948,3.097520,3.064902,2.884723,3.098073,1.495548,1.466794
2,1.546399,1.386535,0.483420,1.558407,1.452383,0.435248,3.055451,3.075237,2.847754,2.805197,1.508741,1.529352
3,1.479263,1.411957,0.474228,1.557954,1.419902,0.437404,3.075902,2.979423,2.876378,2.745932,1.501716,1.508612
4,1.430864,1.394330,0.460893,1.541421,1.398382,0.422379,3.098718,2.957783,2.933187,2.772742,1.493213,1.437839
...,...,...,...,...,...,...,...,...,...,...,...,...
134,1.616677,1.441377,0.452651,1.566328,1.449312,0.429597,3.106595,2.939993,3.090877,3.013240,1.238174,1.247659
135,1.616113,1.433277,0.445019,1.567459,1.449391,0.428806,3.107844,2.938796,3.097528,3.012914,1.252253,1.268736
136,1.616589,1.434066,0.447866,1.572282,1.454354,0.430256,3.111518,2.939261,3.108940,3.022880,1.266450,1.284183
137,1.616517,1.433189,0.448271,1.572723,1.457048,0.429816,3.109683,2.938492,3.112515,3.022459,1.279766,1.285416


In [40]:
cd = get_2D_Coord(data, "nose", 3)

cd2 = get_2D_Coord(data, "left_eye_inner", 3)

vector = get_2D_Vector(data, "nose", "left_eye_inner", 3)
print(cd, "\n", cd2, "\n", vector)

[ 0.21702486 -0.6233452 ] 
 [ 0.19239628 -0.66651171] 
 [0.02462858 0.04316652]
